In [4]:
import requests
import json
import time

In [ ]:
from mem0 import MemoryClient
client = MemoryClient(api_key="<Create your own key>")

In [17]:
# Ollama API endpoint
OLLAMA_URL = "http://localhost:11434/api/generate"

def chat_with_ollama(prompt, conversation_history=""):
    full_prompt = f"{conversation_history}\nUser: {prompt}\nAssistant:"
    
    payload = {
        "model": "llama3",
        "prompt": full_prompt,
        "stream": False
    }
    
    response = requests.post(OLLAMA_URL, json=payload)
    return response.json()["response"]

In [18]:
def store_memory(text, user_id="zyad_user"):
    messages = [{"role": "user", "content": text}]
    client.add(text, user_id=user_id)

In [19]:
def search_memory(query, user_id="zyad_user"):
    return client.search(query, user_id=user_id)

In [20]:
conversation_parts = [
    "Hi! I've been thinking about my friends lately.",
    "My friend Sarah is really into pottery and ceramics. She spends weekends at the local studio.",
    "Tell me about different types of clay used in pottery.",
    "That's interesting! My other friend Mike loves rock climbing and mountaineering.",
    "What's the difference between bouldering and sport climbing?",
    "Speaking of outdoor activities, my friend Lisa is obsessed with bird watching.",
    "She has this amazing collection of vintage binoculars.",
    "What are some rare birds that birdwatchers get excited about?",
    "My friend Tom is a huge jazz enthusiast. He collects vinyl records.",
    "He particularly loves bebop and has original Blue Note pressings.",
    "Tell me about the history of Blue Note Records.",
    "Oh, and my friend Anna is into sustainable gardening and permaculture.",
    "She grows her own vegetables and composts everything.",
    "What are the principles of permaculture design?",
    "My friend David is a coffee roaster who's really passionate about single-origin beans.",
    "He roasts his own beans at home and studies different processing methods.",
    "What's the difference between washed and natural coffee processing?",
    *[f"Let's continue talking about topic number {i}. Can you tell me more details about this subject?" for i in range(20, 50)],
    # Now test memory recall
    "I need help choosing birthday gifts for my friends. Can you help me pick something for Sarah?",
    "What about a gift for Mike?",
    "And what would be good for Lisa?",
    "Any ideas for Tom's gift?",
    "What about Anna?",
    "And finally, what would David like?"
]


In [21]:
def run_memory_test():
    conversation_history = ""
    user_id = f"zyad_user_{int(time.time())}"
    
    print("=== RUNNING MEMORY TEST ===\n")
    
    # Run the long conversation
    for i, user_input in enumerate(conversation_parts):
        print(f"Turn {i+1}: {user_input}")
        
        # Get LLM response
        response = chat_with_ollama(user_input, conversation_history)
        print(f"Response: {response[:100]}...")
        
        # Store conversation turn in memory
        messages = [
            {"role": "user", "content": user_input},
            {"role": "assistant", "content": response}
        ]
        client.add(messages, user_id=user_id)
        
        # Update conversation history (keep it manageable)
        conversation_history += f"\nUser: {user_input}\nAssistant: {response}"
        if len(conversation_history) > 4000:  # Trim if too long
            conversation_history = conversation_history[-3000:]
        
        print("-" * 50)
        time.sleep(0.5)  # Be nice to the APIs
    
    # Test memory recall
    print("\n=== TESTING MEMORY RECALL ===")
    friends = ["Sarah", "Mike", "Lisa", "Tom", "Anna", "David"]
    
    for friend in friends:
        print(f"\nSearching memories about {friend}:")
        memories = search_memory(f"{friend} interests hobbies", user_id)
        for memory in memories:
            print(f"  - {memory['memory']}")

In [22]:
if __name__ == "__main__":
    run_memory_test()

=== RUNNING MEMORY TEST ===

Turn 1: Hi! I've been thinking about my friends lately.
Response: That's great! Thinking about your friends can be a wonderful way to reflect on the importance of rel...
--------------------------------------------------
Turn 2: My friend Sarah is really into pottery and ceramics. She spends weekends at the local studio.
Response: That sounds like a fun interest! Pottery and ceramics can be such a relaxing and creative outlet. Wh...
--------------------------------------------------
Turn 3: Tell me about different types of clay used in pottery.
Response: There are many different types of clay used in pottery, each with its own unique characteristics and...
--------------------------------------------------
Turn 4: That's interesting! My other friend Mike loves rock climbing and mountaineering.
Response: That sounds like an exciting interest! Rock climbing and mountaineering can be such thrilling hobbie...
--------------------------------------------------
T

KeyboardInterrupt: 

In [ ]:
# Test memory recall
print("\n=== TESTING MEMORY RECALL ===")
friends = ["Sarah", "Mike", "Lisa", "Tom", "Anna", "David"]

for friend in friends:
    print(f"\nSearching memories about {friend}:")
    memories = search_memory(f"{friend} interests hobbies", "<Your-user>")
    for memory in memories:
        print(f"  - {memory['memory']}")


=== TESTING MEMORY RECALL ===

Searching memories about Sarah:
  - Has a friend named Sarah who is really into pottery and ceramics
  - Sarah spends weekends at the local studio
  - Has a friend named Lisa who is passionate about bird watching
  - Has a friend named Anna who is into sustainable gardening and permaculture
  - User is interested in rare birds that excite birdwatchers
  - Has a friend named Mike who loves rock climbing and mountaineering
  - Has a friend named Tom who is a jazz enthusiast and collects original Blue Note bebop vinyl records
  - Lisa has a collection of vintage binoculars
  - Tom collects vinyl records
  - Tom particularly loves bebop and has original Blue Note pressings

Searching memories about Mike:
  - Has a friend named Mike who loves rock climbing and mountaineering
  - Tom collects vinyl records
  - Has a friend named Tom who is a jazz enthusiast and collects original Blue Note bebop vinyl records
  - Has a friend named Sarah who is really into pott